# Habitat suitability for Red Pandas -- Dataset Synthesis

# Synthetic Habitat Suitability Dataset Creation Summary

A synthetic dataset has been created to model the habitat suitability for pandas. The dataset comprises 100,000 samples, each with the following variables:

- `Altitude`: Generated from a uniform distribution ranging from 1200 to 3400 meters.
- `Distance_from_Human_Paths`: Generated from an exponential distribution, with a mean distance of 500 meters.
- `Livestock_Density`: Generated from a gamma distribution to represent skewed data, with a shape parameter of 2 and a scale of 0.5.
- `Vegetation_Diversity_Index`: Generated from a uniform distribution between 0 and 1.
- `Water_Source_Availability`: Binary variable generated with a preference for '1' (70%) over '0' (30%).
- `Human_Disturbance_Index`: Generated from a uniform distribution between 0 and 1.
- `Slope`: Generated from a uniform distribution between 0 and 30 degrees.
- `Annual_Rainfall`: Generated from a normal distribution centered at 1500 mm with a standard deviation of 250 mm.

Correlations and noise have been introduced to reflect ecological relationships:

1. **Altitude and Bamboo Coverage**: A positive linear relationship with 10% noise added.
2. **Livestock Density and Vegetation Diversity Index**: An inverse relationship modified by the exponential of negative livestock density, with 5% noise added.
3. **Water Source Availability and Annual Rainfall**: A conditional relationship where higher rainfall increases the likelihood of water availability, with 10% noise introduced by inverting the availability status.
4. **Distance from Human Paths and Human Disturbance Index**: An inverse relationship where greater distance from paths lowers the disturbance index, with 10% noise added.
5. **Slope and Bamboo Coverage**: A conditional relationship where slopes under 20 degrees do not affect bamboo coverage, while steeper slopes reduce coverage by half, with 10% noise added.

The `Suitable` variable is defined based on a K-Means Clustering Algortihm, and artificially manipulated to have 30% of rows identified as suitable.

Lastly, `Mean_Annual_Temperature` is derived from altitude with a simple linear relationship and added noise, to reflect the decrease in temperature with increasing altitude.

The dataset synthesis process includes random noise to simulate natural variability and to prevent perfect correlations, aiming to create a realistic dataset for training predictive models.


**Features values generation**

In [2]:
import pandas as pd
import numpy as np

# Set the random seed for reproducibility
np.random.seed(42)

# Number of samples in the dataset
n_samples = 100000

# Generate the synthetic data
data = pd.DataFrame({
    'Altitude': np.random.uniform(1200, 3400, n_samples),  # Uniform distribution
    'Distance_from_Human_Paths': np.random.exponential(500, n_samples),  # Exponential distribution
    'Livestock_Density': np.random.gamma(2, 0.5, n_samples),  # Gamma distribution for skewed data
    'Vegetation_Diversity_Index': np.random.uniform(0, 1, n_samples),  # Uniform distribution
    # This one is not good, needs to be corrected
    'Water_Source_Availability': np.random.choice([0, 1], n_samples, p=[0.3, 0.7]),  # Binary, with more 1s
    'Human_Disturbance_Index': np.random.uniform(0, 1, n_samples),  # Uniform distribution
    'Slope': np.random.uniform(0, 30, n_samples),  # Uniform distribution
    'Annual_Rainfall': np.random.normal(1500, 250, n_samples)  # Normal distribution
})

# Add noise function
def add_noise(factor, size):
    return np.random.normal(1, factor, size)

# Correlation 1: Altitude and Bamboo Coverage (with noise)
data['Bamboo_Coverage'] = np.interp(data['Altitude'], [1200, 3400], [0, 100])
data['Bamboo_Coverage'] *= add_noise(0.1, n_samples)  # Adding 10% noise

# Correlation 2: Livestock Density and Vegetation Diversity Index (with noise)
data['Vegetation_Diversity_Index'] *= np.exp(-data['Livestock_Density'])
data['Vegetation_Diversity_Index'] *= add_noise(0.05, n_samples)  # Adding 5% noise

# Correlation 3: Water Source Availability and Annual Rainfall (with noise)
data['Water_Source_Availability'] = np.where(data['Annual_Rainfall'] > data['Annual_Rainfall'].mean(), 1, 0)
data['Water_Source_Availability'] = np.where(np.random.rand(n_samples) < 0.1, 1 - data['Water_Source_Availability'], data['Water_Source_Availability'])  # Inverting 10% to add noise

# Correlation 4: Distance from Human Paths and Human Disturbance Index (with noise)
data['Human_Disturbance_Index'] = np.interp(data['Distance_from_Human_Paths'], [0, max(data['Distance_from_Human_Paths'])], [1, 0])
data['Human_Disturbance_Index'] *= add_noise(0.1, n_samples)  # Adding 10% noise

# Correlation 5: Slope and Bamboo Coverage (with noise)
data['Bamboo_Coverage'] *= np.where(data['Slope'] < 20, 1, 0.5)  # Reduce bamboo coverage on steep slopes
data['Bamboo_Coverage'] *= add_noise(0.1, n_samples)  # Adding 10% noise

# Mean Annual Temperature is derived with some noise, considering its correlation with Altitude
data['Mean_Annual_Temperature'] = 20 - (data['Altitude'] / 200)  # Simplified linear relation
data['Mean_Annual_Temperature'] += np.random.normal(0, 1, n_samples)  # Adding noise

# Output the first few rows of the dataset
data.head()

,Altitude,Distance_from_Human_Paths,Livestock_Density,Vegetation_Diversity_Index,Water_Source_Availability,Human_Disturbance_Index,Slope,Annual_Rainfall,Bamboo_Coverage,Mean_Annual_Temperature
0,2023.988261,434.678578,0.634095,0.319653,0,1.025552,1.902428,1364.685583,30.270545,9.912647
1,3291.571474,374.299977,0.132676,0.604097,1,1.032245,16.484248,1616.727901,71.382519,1.701986
2,2810.386672,216.189749,0.477984,0.157106,0,0.963946,29.466434,1373.229653,31.969303,6.258988
3,2517.048665,339.831903,0.386706,0.560503,1,1.142016,9.966878,1747.708413,50.076009,7.423174
4,1543.241009,227.141237,0.654869,0.349722,1,1.050573,10.647234,1500.073131,17.449099,12.331676


In [2]:
import pandas as pd
import numpy as np

# Set the random seed for reproducibility
np.random.seed(42)

# Number of samples in the dataset
n_samples = 100000

# Generate the synthetic data
data = pd.DataFrame({
    'Altitude': np.random.uniform(1200, 3400, n_samples),  # Uniform distribution
    'Distance_from_Human_Paths': np.random.exponential(500, n_samples),  # Exponential distribution
    'Livestock_Density': np.random.gamma(2, 0.5, n_samples),  # Gamma distribution for skewed data
    'Vegetation_Diversity_Index': np.random.uniform(0, 1, n_samples),  # Uniform distribution
    'Water_Source_Availability': np.random.choice([0, 1], n_samples, p=[0.3, 0.7]),  # Binary, with more 1s
    'Human_Disturbance_Index': np.random.uniform(0, 1, n_samples),  # Uniform distribution
    'Slope': np.random.uniform(0, 30, n_samples),  # Uniform distribution
    'Annual_Rainfall': np.random.normal(1500, 250, n_samples)  # Normal distribution
})

# Noise and transformations functions
def add_noise(factor, size):
    return np.random.normal(1, factor, size)

def add_complex_noise(data, feature, noise_level, correlation_feature=None, correlation_func=None):
    noise = np.random.normal(1, noise_level, data.shape[0])
    if correlation_feature is not None and correlation_func is not None:
        correlation = correlation_func(data[correlation_feature])
        return data[feature] * noise * correlation
    return data[feature] * noise

def add_outliers(data, feature, proportion, scale):
    indices = np.random.choice(data.index, size=int(proportion * len(data)), replace=False)
    outliers = np.random.normal(scale=np.std(data[feature]) * scale, size=indices.shape[0])
    data.loc[indices, feature] += outliers

# Correlations functions
def correlate_with_altitude(data, feature_name, base_value, scale_factor, noise_std=0):
    # Elevation-dependent feature with added noise
    data[feature_name] = base_value - (data['Altitude'] / scale_factor)
    data[feature_name] += np.random.normal(0, noise_std, len(data))
    data[feature_name] = data[feature_name].clip(lower=0)  # Ensure values are not negative

def correlate_with_water_source(data, feature_name, base_value, variation_factor, noise_std=0):
    # Feature varies with water source availability
    data[feature_name] = data['Water_Source_Availability'].apply(lambda x: base_value + np.random.normal(variation_factor if x == 1 else -variation_factor, noise_std))

def correlate_with_human_disturbance(data, feature_name, base_value, scale_factor, noise_std=0):
    # Feature decreases with more human disturbance
    data[feature_name] = base_value - (data['Human_Disturbance_Index'] * scale_factor)
    data[feature_name] += np.random.normal(0, noise_std, len(data))
    data[feature_name] = data[feature_name].clip(lower=0)

# New Correlations
def correlate_slope_with_water(data):
    data['Water_Source_Availability'] = np.where(data['Slope'] > 15, 0, data['Water_Source_Availability'])

def correlate_distance_with_vegetation(data):
    data['Vegetation_Diversity_Index'] *= np.exp(-data['Distance_from_Human_Paths'] / 1000)

def correlate_livestock_with_human_disturbance(data):
    data['Livestock_Density'] *= 1 + (data['Human_Disturbance_Index'] * 0.5)

def correlate_altitude_with_bamboo(data):
    data['Bamboo_Coverage'] = np.interp(data['Altitude'], [1200, 3400], [100, 0])
    data['Bamboo_Coverage'] *= add_noise(0.25, n_samples)

def correlate_rainfall_with_vegetation(data):
    # Assuming that a certain level of rainfall is beneficial but too much might be detrimental
    optimal_rainfall = 1500
    data['Vegetation_Diversity_Index'] *= np.where(
        data['Annual_Rainfall'] < optimal_rainfall,
        # Scale positively with rainfall up to the optimal point
        data['Annual_Rainfall'] / optimal_rainfall,
        # Beyond the optimal point, start scaling negatively
        optimal_rainfall / data['Annual_Rainfall']
    )

# Correlation between Altitude and Livestock Density
def correlate_altitude_with_livestock(data):
    # Assuming livestock density decreases by 0.2% for each meter increase in altitude
    data['Livestock_Density'] *= (1 - data['Altitude'] * 0.002 / 100)

# Correlation between Human Disturbance Index and Water Source Availability
def correlate_human_disturbance_with_water(data):
    # Assuming the presence of water is reduced by 50% in areas with high human disturbance
    data['Water_Source_Availability'] *= np.where(
        data['Human_Disturbance_Index'] > 0.5,
        0.5,
        1
    )

# Applying new correlations
correlate_rainfall_with_vegetation(data)
correlate_altitude_with_livestock(data)
correlate_human_disturbance_with_water(data)
correlate_slope_with_water(data)
correlate_distance_with_vegetation(data)
correlate_livestock_with_human_disturbance(data)
correlate_altitude_with_bamboo(data)
correlate_with_altitude(data, 'Mean_Annual_Temperature', base_value=30, scale_factor=300, noise_std=1)
correlate_with_water_source(data, 'Vegetation_Diversity_Index', base_value=0.5, variation_factor=0.3, noise_std=0.05)
correlate_with_human_disturbance(data, 'Livestock_Density', base_value=5, scale_factor=3, noise_std=0.1)


# Applying correlations and noise
data['Bamboo_Coverage'] = np.interp(data['Altitude'], [1200, 3400], [0, 100])
data['Bamboo_Coverage'] *= add_noise(0.25, n_samples)

data['Vegetation_Diversity_Index'] *= np.exp(-data['Livestock_Density'])
data['Vegetation_Diversity_Index'] *= add_noise(0.1, n_samples)

data['Water_Source_Availability'] = np.where(data['Annual_Rainfall'] > data['Annual_Rainfall'].mean(), 1, 0)
data['Water_Source_Availability'] = np.where(np.random.rand(n_samples) < 0.1, 1 - data['Water_Source_Availability'], data['Water_Source_Availability'])

data['Human_Disturbance_Index'] = np.interp(data['Distance_from_Human_Paths'], [0, max(data['Distance_from_Human_Paths'])], [1, 0])
data['Human_Disturbance_Index'] *= add_noise(0.25, n_samples)

data['Bamboo_Coverage'] *= np.where(data['Slope'] < 20, 1, 0.5)
data['Bamboo_Coverage'] *= add_noise(0.3, n_samples)

data['Mean_Annual_Temperature'] = 20 - (data['Altitude'] / 200)
data['Mean_Annual_Temperature'] += np.random.normal(0, 1, n_samples)

# Adding complex noise
data['Annual_Rainfall'] = add_complex_noise(data, 'Annual_Rainfall', 0.1, 'Altitude', lambda x: np.exp(-x/2000))

# Adding outliers
add_outliers(data, 'Annual_Rainfall', 0.01, 10)

In [3]:
data

,Altitude,Distance_from_Human_Paths,Livestock_Density,Vegetation_Diversity_Index,Water_Source_Availability,Human_Disturbance_Index,Slope,Annual_Rainfall,Bamboo_Coverage,Mean_Annual_Temperature
0,2023.988261,434.678578,3.446700,0.008606,0,0.958644,1.902428,481.346699,40.438091,9.787795
1,3291.571474,374.299977,2.873850,0.011834,1,0.838599,16.484248,272.127408,55.983789,5.280065
2,2810.386672,216.189749,4.991707,0.001131,0,1.140845,29.466434,276.755018,44.974754,6.667004
3,2517.048665,339.831903,3.594812,0.021270,1,1.043919,9.966878,553.044339,88.458894,8.003624
4,1543.241009,227.141237,3.313984,0.006689,1,1.010359,10.647234,677.418356,7.053957,13.797550
...,...,...,...,...,...,...,...,...,...,...
99995,2943.070627,486.628948,3.810608,0.004137,0,0.849407,27.083635,280.787687,22.979504,6.265352
99996,2914.356369,496.073871,4.235591,0.009070,1,0.847571,10.520047,379.771440,116.050033,5.233896
99997,2683.797496,106.722664,4.845199,0.006228,1,0.899508,11.544332,382.667603,71.238266,4.971030
99998,2298.783938,529.334414,3.380380,0.009181,0,0.976766,23.171634,504.726645,17.879073,7.906713


In [4]:
data

,Altitude,Distance_from_Human_Paths,Livestock_Density,Vegetation_Diversity_Index,Water_Source_Availability,Human_Disturbance_Index,Slope,Annual_Rainfall,Bamboo_Coverage,Mean_Annual_Temperature
0,2023.988261,434.678578,3.446700,0.008606,0,0.958644,1.902428,481.346699,40.438091,9.787795
1,3291.571474,374.299977,2.873850,0.011834,1,0.838599,16.484248,272.127408,55.983789,5.280065
2,2810.386672,216.189749,4.991707,0.001131,0,1.140845,29.466434,276.755018,44.974754,6.667004
3,2517.048665,339.831903,3.594812,0.021270,1,1.043919,9.966878,553.044339,88.458894,8.003624
4,1543.241009,227.141237,3.313984,0.006689,1,1.010359,10.647234,677.418356,7.053957,13.797550
...,...,...,...,...,...,...,...,...,...,...
99995,2943.070627,486.628948,3.810608,0.004137,0,0.849407,27.083635,280.787687,22.979504,6.265352
99996,2914.356369,496.073871,4.235591,0.009070,1,0.847571,10.520047,379.771440,116.050033,5.233896
99997,2683.797496,106.722664,4.845199,0.006228,1,0.899508,11.544332,382.667603,71.238266,4.971030
99998,2298.783938,529.334414,3.380380,0.009181,0,0.976766,23.171634,504.726645,17.879073,7.906713


**'Suitable' column estimation -- KMeans**

Artificially manipulated to get 30% of environments as suitable.

In [5]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# Use KMeans clustering
kmeans = KMeans(n_clusters=2, random_state=42)
clusters = kmeans.fit_predict(data_scaled)

# Add the cluster information to the original data
data['Cluster'] = clusters

# Identify the centroids for each cluster
centroids = kmeans.cluster_centers_

# Calculate the distance of each point to the centroids
distances = kmeans.transform(data_scaled)

# Get the distances to the "farthest" centroid (assuming cluster 0 is 'unsuitable', cluster 1 is 'suitable')
data['Distance_to_Unsuitable'] = distances[:, 0]
data['Distance_to_Suitable'] = distances[:, 1]

# Determine the 30th percentile threshold for the distance to the 'suitable' centroid
threshold = np.percentile(data['Distance_to_Suitable'], 70)  # since we want the closest 30%

# Label the data based on the threshold
data['Suitable'] = (data['Distance_to_Suitable'] <= threshold).astype(int)

# Now, 30% of your data should be labeled as suitable
print(data['Suitable'].value_counts(normalize=True))  # Check the proportion of '1's and '0's

# Drop the columns not needed anymore
data.drop(['Cluster', 'Distance_to_Unsuitable', 'Distance_to_Suitable'], axis=1, inplace=True)

c:\Users\giorg\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


1    0.7
0    0.3
Name: Suitable, dtype: float64


In [6]:
data.Suitable.value_counts()

1    70000
0    30000
Name: Suitable, dtype: int64

In [9]:
data.to_csv('data/habitat-suitability.csv', index=False)

**Baseline Logistic Regression model**

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Step 1: Prepare the data
# Features matrix 'X' and target variable 'y'
X = data.drop('Suitable', axis=1)
y = data['Suitable']

# Step 2: Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 3: Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 4: Initialize and train the Logistic Regression model
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train_scaled, y_train)

# Step 5: Predictions and evaluation
y_pred = log_reg.predict(X_test_scaled)

# Model evaluation
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.90

Confusion Matrix:
 [[ 7326  1658]
 [ 1451 19565]]

Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.82      0.82      8984
           1       0.92      0.93      0.93     21016

    accuracy                           0.90     30000
   macro avg       0.88      0.87      0.88     30000
weighted avg       0.90      0.90      0.90     30000



# Transportation Network Structure Dataset Synthesis 

In [8]:
import random 

def generate_optimization_dataset(n_origins, n_transshipments, n_destinations, total_supply,
                                  min_trans_cost = 1, max_trans_cost = 100):
    
    X = n_origins
    Y = n_destinations
    Z = n_transshipments
    
    ## Transportation Costs
    # Origin to transshipment
    origin_to_transshipment_cost = {}
    for i in range(X):
        for k in range(Z):
            key = f"{i+1}-{k+1}"
            cost = random.randint(min_trans_cost, max_trans_cost)  # Random uniform
            origin_to_transshipment_cost[key] = cost

    # Transshipment to destination
    transshipment_to_destination_cost = {}
    for k in range(Z):
        for j in range(Y):
            key = f"{k+1}-{j+1}"
            cost = random.randint(min_trans_cost, max_trans_cost)  # Random uniform
            transshipment_to_destination_cost[key] = cost


    ## Capacities
    # Transhipment capacity

    min_capacity_hub = total_supply // Z # To be refined
    max_capacity_hub = total_supply // Z + 1000

    transshipment_capacity = {}
    for k in range(Z):
        capacity = random.randint(min_capacity_hub, max_capacity_hub)  # Random capacity between 1000 and 5000
        transshipment_capacity[f"{k+1}"] = capacity

    # Destination capacity

    min_capacity_destination = total_supply // Y # To be refined
    max_capacity_destination = total_supply
    destination_capacity = {}
    for j in range(Y):
        capacity = random.randint(min_capacity_destination, max_capacity_destination)
        destination_capacity[f"{j+1}"] = capacity


    ## Supply of origin nodes
    origin_supply = {}

    proportions = np.random.dirichlet(np.ones(X),size=1)

    # Distribute the demand across all origin supply nodes
    for i in range(X):
        origin_supply[f"{i+1}"] = int(proportions[0][i] * total_supply)
    
    return [origin_to_transshipment_cost, transshipment_to_destination_cost, transshipment_capacity, destination_capacity, origin_supply]


In [94]:
network = generate_optimization_dataset(n_origins=30, n_transshipments=10, n_destinations=10, total_supply=10000)

In [95]:
origin_to_transshipment_cost = network[0]
origin_to_transshipment_cost_df = pd.DataFrame(list(origin_to_transshipment_cost.items()), columns=['Key', 'Value'])
origin_to_transshipment_cost_df['Origin'] = origin_to_transshipment_cost_df['Key'].str.split('-').str[0]
origin_to_transshipment_cost_df['Hub'] = origin_to_transshipment_cost_df['Key'].str.split('-').str[1]
origin_to_transshipment_cost_df = origin_to_transshipment_cost_df.drop('Key', axis=1)


transshipment_to_destination_cost = network[1]
transshipment_to_destination_cost_df = pd.DataFrame(list(transshipment_to_destination_cost.items()), columns=['Key', 'Value'])
transshipment_to_destination_cost_df['Hub'] = transshipment_to_destination_cost_df['Key'].str.split('-').str[0]
transshipment_to_destination_cost_df['Destination'] = transshipment_to_destination_cost_df['Key'].str.split('-').str[1]
transshipment_to_destination_cost_df = transshipment_to_destination_cost_df.drop('Key', axis=1)

transshipment_capacity = network[2] 
transshipment_capacity_df = pd.DataFrame(list(transshipment_capacity.items()), columns=['Hub', 'Value'])

destination_capacity = network[3]
destination_capacity_df = pd.DataFrame(list(destination_capacity.items()), columns=['Destination', 'Value'])

origin_supply = network[4]
origin_supply_df = pd.DataFrame(list(origin_supply.items()), columns=['Origin', 'Value'])

In [96]:
origin_to_transshipment_cost_df.to_csv('data/opti-origin_to_transshipment_cost.csv', index=False)
transshipment_to_destination_cost_df.to_csv('data/opti-transshipment_to_destination_cost.csv', index=False)  
transshipment_capacity_df.to_csv('data/opti-transshipment_capacity.csv', index=False)
destination_capacity_df.to_csv('data/opti-destination_capacity.csv', index=False)
origin_supply_df.to_csv('data/opti-origin_supply.csv', index=False)